### Decision Tree

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
# Open the .csv files
X_train = pd.read_csv('/Users/arnav/Desktop/MachineLearning/ML_CSE343 Project/train_word2vec.csv')
X_test = pd.read_csv('/Users/arnav/Desktop/MachineLearning/ML_CSE343 Project/test_word2vec.csv')

X_train = X_train.drop(['Unnamed: 0'], axis=1)
X_test = X_test.drop(['Unnamed: 0'], axis=1)

# Check the shape of the data
print(X_train.shape)
print(X_test.shape)

(22500, 512)
(7500, 512)


In [3]:
# Extract the features into a numpy array
X_train = X_train.values
X_test = X_test.values

# Open the labels
y_train_df = pd.read_csv('/Users/arnav/Desktop/MachineLearning/ML_CSE343 Project/CSE343-ML-Project/Data/Preprocessed-Data/train_labels.csv')
y_test_df = pd.read_csv('/Users/arnav/Desktop/MachineLearning/ML_CSE343 Project/CSE343-ML-Project/Data/Preprocessed-Data/test_labels.csv')

# Check the shape of the labels
print(y_train_df.shape)
print(y_test_df.shape)

(22500, 1)
(7500, 1)


In [4]:
# Convert class to non_suicide = 0 and suicide = 1
y_train_temp = y_train_df['class'].values
y_test_temp = y_test_df['class'].values

# Check the shape of the labels
print(y_train_temp.shape)
print(y_test_temp.shape)

y_train = np.array([0 if label == "non-suicide" else 1 for label in y_train_temp])
y_test = np.array([0 if label == "non-suicide" else 1 for label in y_test_temp])

# Check the shape of the labels
print(y_train.shape)
print(y_test.shape)

(22500,)
(7500,)
(22500,)
(7500,)


In [5]:
# Standard Preprocessing
# Check the mean and standard deviation of the data
print(np.mean(X_train), np.std(X_train))

0.0029877624999040935 0.18965241040518302


### Running without Hyperparameter Tuning

In [6]:
# Decision Tree Classifier
dtclf = DecisionTreeClassifier()

# Fit the model
dtclf.fit(X_train, y_train)

# Predict on the test data
y_pred = dtclf.predict(X_test)


In [7]:
# Print the accuracy score
print("Accuracy Score:", accuracy_score(y_test, y_pred))

# Print the confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Print AUC score
print("AUC Score:", roc_auc_score(y_test, y_pred))

# Print the classification report
print("Classification Report:\n", classification_report(y_test, y_pred))

# Print weighted F1-score
print("Weighted F1-score:", f1_score(y_test, y_pred, average='weighted'))

Accuracy Score: 0.836
Confusion Matrix:
 [[3035  673]
 [ 557 3235]]
AUC Score: 0.8358061768601588
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.82      0.83      3708
           1       0.83      0.85      0.84      3792

    accuracy                           0.84      7500
   macro avg       0.84      0.84      0.84      7500
weighted avg       0.84      0.84      0.84      7500

Weighted F1-score: 0.8359323109766945


### Grid Search

In [10]:
# define hyperparameters grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [11]:
# Initialize Decision Tree Classifier
clf = DecisionTreeClassifier()

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(clf, param_grid, cv=5, verbose=2, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=  36.2s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=  37.8s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=random; total time=  38.8s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=  41.2s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=  42.3s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=  42.4s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=  43.6s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, splitter=best; total time=  46.3s
[C

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30, 40, 50],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'splitter': ['best', 'random']},
             verbose=2)

In [15]:
# Print the best parameters
print(grid_search.best_params_)

{'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'splitter': 'best'}


In [8]:
# Best Decision Tree Classifier
dtclf_best = DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_leaf=1, min_samples_split=5,
                                    splitter='best')

# Fit the model
dtclf_best.fit(X_train, y_train)

# Predict on the test data
y_pred = dtclf_best.predict(X_test)

In [9]:
# Print the accuracy score
print("Best Decision Tree Classifier: Test Data")
print("Accuracy Score:", accuracy_score(y_test, y_pred))

# Print the confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Print AUC score
print("AUC Score:", roc_auc_score(y_test, y_pred))

# Print the classification report
print("Classification Report:\n", classification_report(y_test, y_pred))

# Print weighted F1-score
print("Weighted F1-score:", f1_score(y_test, y_pred, average='weighted'))

Best Decision Tree Classifier: Test Data
Accuracy Score: 0.8672
Confusion Matrix:
 [[3055  653]
 [ 343 3449]]
AUC Score: 0.8667203480671282
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.82      0.86      3708
           1       0.84      0.91      0.87      3792

    accuracy                           0.87      7500
   macro avg       0.87      0.87      0.87      7500
weighted avg       0.87      0.87      0.87      7500

Weighted F1-score: 0.8669108430298413


In [10]:
# Traininig Data
# Predict on the train data
y_pred = dtclf_best.predict(X_train)

# Print the accuracy score
print("Best Decision Tree Classifier: Training Data")
print("Accuracy Score:", accuracy_score(y_train, y_pred))

# Print the confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred))

# Print AUC score
print("AUC Score:", roc_auc_score(y_train, y_pred))

# Print the classification report
print("Classification Report:\n", classification_report(y_train, y_pred))

# Print weighted F1-score
print("Weighted F1-score:", f1_score(y_train, y_pred, average='weighted'))

Best Decision Tree Classifier: Training Data
Accuracy Score: 0.9480444444444445
Confusion Matrix:
 [[10478   814]
 [  355 10853]]
AUC Score: 0.9481198813508795
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.93      0.95     11292
           1       0.93      0.97      0.95     11208

    accuracy                           0.95     22500
   macro avg       0.95      0.95      0.95     22500
weighted avg       0.95      0.95      0.95     22500

Weighted F1-score: 0.9480267746472785


### Principal Component Analysis

In [11]:
pca = PCA(n_components=0.75)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [12]:
print(X_train_pca.shape)

(22500, 13)


In [13]:
#Run the model on the PCA data

# Initialize Decision Tree Classifier
dtclf_pca = DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_leaf=1, min_samples_split=5,
                                    splitter='best')

dtclf_pca.fit(X_train_pca, y_train)

y_pred_test_pca = dtclf_pca.predict(X_test_pca)

In [14]:
#Print the accuracy score
print("Decision Tree Classifier with PCA : Test Data")
print("Accuracy Score:", accuracy_score(y_test, y_pred_test_pca))

# Print the confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test_pca))

# Print AUC score
print("AUC Score:", roc_auc_score(y_test, y_pred_test_pca))

# Print the classification report
print("Classification Report:\n", classification_report(y_test, y_pred_test_pca))

# Print weighted F1-score
print("Weighted F1-score:", f1_score(y_test, y_pred_test_pca, average='weighted'))


Decision Tree Classifier with PCA : Test Data
Accuracy Score: 0.8598666666666667
Confusion Matrix:
 [[3059  649]
 [ 402 3390]]
AUC Score: 0.8594801865279315
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.82      0.85      3708
           1       0.84      0.89      0.87      3792

    accuracy                           0.86      7500
   macro avg       0.86      0.86      0.86      7500
weighted avg       0.86      0.86      0.86      7500

Weighted F1-score: 0.8596625912953763


In [15]:
# Training Data
# Predict on the training data
y_pred_train_pca = dtclf_pca.predict(X_train_pca)

# Print the accuracy score
print("Decision Tree Classifier with PCA: Training Data")
print("Accuracy Score:", accuracy_score(y_train, y_pred_train_pca))

# Print the confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred_train_pca))

# Print AUC score
print("AUC Score:", roc_auc_score(y_train, y_pred_train_pca))

# Print the classification report
print("Classification Report:\n", classification_report(y_train, y_pred_train_pca))

# Print weighted F1-score
print("Weighted F1-score:", f1_score(y_train, y_pred_train_pca, average='weighted'))

Decision Tree Classifier with PCA: Training Data
Accuracy Score: 0.9249333333333334
Confusion Matrix:
 [[10195  1097]
 [  592 10616]]
AUC Score: 0.9250160808167235
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.90      0.92     11292
           1       0.91      0.95      0.93     11208

    accuracy                           0.92     22500
   macro avg       0.93      0.93      0.92     22500
weighted avg       0.93      0.92      0.92     22500

Weighted F1-score: 0.9249017972579711
